In [ ]:
# from keras.applications.vgg16 import (VGG16, preprocess_input, decode_predictions)
# from keras.applications.vgg19 import (VGG19, preprocess_input, decode_predictions)
from keras.applications.resnet50 import (ResNet50, preprocess_input, decode_predictions)
from keras.preprocessing import image
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import cv2
import os

from keras.models import load_model

from art.attacks import DeepFool
from art.attacks import FastGradientMethod
from art.attacks.evasion.zoo import ZooAttack
from art.attacks.evasion.elastic_net import ElasticNet
from art.classifiers import KerasClassifier
from art.attacks.evasion.carlini import CarliniL2Method, CarliniLInfMethod

from scipy import ndimage
import imageio

# In[2]:


def load_image(path):
    img_path = path
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * tf.cast(op.inputs[0] > 0., dtype)

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def grad_cam(model, x, category_index, layer_name):
    """
    Args:
       model: model
       x: image input
       category_index: category index
       layer_name: last convolution layer name
    """
    # get category loss
    class_output = model.output[:, category_index]

    # layer output
    convolution_output = model.get_layer(layer_name).output
    # get gradients
    grads = K.gradients(class_output, convolution_output)[0]
    # get convolution output and gradients for input
    gradient_function = K.function([model.input], [convolution_output, grads])

    output, grads_val = gradient_function([x])
    output, grads_val = output[0], grads_val[0]

    # avg
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # create heat map
    cam = cv2.resize(cam, (x.shape[1], x.shape[2]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    # Return to BGR [0..255] from the preprocessed image
    image_rgb = x[0, :]
    image_rgb -= np.min(image_rgb)
    image_rgb = np.minimum(image_rgb, 255)

    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image_rgb)
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap

def compile_saliency_function(model, activation_layer):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])


# In[3]:


def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model


In [ ]:
# x_val = np.load('VGG16_val_215.npy',mmap_mode = 'c')
# y_val = np.load('VGG16_y_val_215.npy',mmap_mode = 'r')
x_val = np.load('ResNet50_val_226.npy',mmap_mode = 'c')
y_val = np.load('ResNet50_y_val_226.npy',mmap_mode = 'r')
# min_, max_ = 0,255
x_val = x_val/255
min_, max_ = 0,1

# model = VGG16(weights='imagenet')
model = ResNet50(weights='imagenet')
model.summary()

imgAdv = np.load('cw_resnet226_x.npy',mmap_mode = 'c')
imgAdvPredictions = np.load('cw_resnet226_y.npy',mmap_mode = 'r')

cmp = (np.argmax(imgAdvPredictions, axis=1))
attackSuccessIndex = []
for i in range(len(imgAdvPredictions)):
    if cmp[i] != y_val[i]:
        attackSuccessIndex.append(i)
        
print (attackSuccessIndex)
print (len(attackSuccessIndex))

In [ ]:
print('normal')
successNum = [0,0,0,0,0,0]

for i in range(len(attackSuccessIndex)):
    img = x_val[i]*255

    img1 = np.expand_dims(img, axis=0)
    img1 = img1.astype('float32')

    img1 = preprocess_input(img1)

    img4predict = np.copy(img1)

    predictions = model.predict(img4predict)
    predicted_class = np.argmax(predictions)
    
    # cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "block5_conv3")
    cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "res5c_branch2c")
    
    combineImg = np.zeros((6, 224, 224, 3), dtype=np.float32)
    need_test = [0.00000001,0.1,0.2,0.3,0.4,0.5]
    for j in range(len(need_test)):
        combineImg[j] = cam_image * need_test[j] + img

        #*******
        img2 = np.expand_dims(combineImg[j], axis=0)
        # img2 = np.expand_dims(combineImg[j][:,:,::-1], axis=0)
        img2 = img2.astype('float32')
        
        #*******
        img2 = preprocess_input(img2)

        #*******
        combine4predict = np.copy(img2)

        predictions_Combine = model.predict(combine4predict)
        # predictions_Combine = model.predict(combine4predict[...,::-1])
        predicted_class_Combine = np.argmax(predictions_Combine)

        if predicted_class == predicted_class_Combine:
            successNum[j] += 1
    
for z in range(len(need_test)):
    print ('Detection Accuracy is: %.4f'  %(successNum[z]/len(attackSuccessIndex)))

    
    
    
successDetectionNum = [0,0,0,0,0,0]

for i in range(len(attackSuccessIndex)):
    # img = imgAdv[attackSuccessIndex[i]]
    img = imgAdv[attackSuccessIndex[i]]*255

    img1 = np.expand_dims(img, axis=0)
    img1 = img1.astype('float32')

    img1 = preprocess_input(img1)

    img4predict = np.copy(img1)

    predictions = model.predict(img4predict)
    predicted_class = np.argmax(predictions)
    
    # cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "block5_conv3")
    cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "res5c_branch2c")

    combineImg = np.zeros((6, 224, 224, 3), dtype=np.float32)
    need_test = [0.00000001,0.1,0.2,0.3,0.4,0.5]
    for j in range(len(need_test)):
        combineImg[j] = cam_image * need_test[j] + img

        #*******
        img2 = np.expand_dims(combineImg[j], axis=0)
        # img2 = np.expand_dims(combineImg[j][:,:,::-1], axis=0)
        img2 = img2.astype('float32')
        
        #*******
        img2 = preprocess_input(img2)

        #*******
        combine4predict = np.copy(img2)

        predictions_Combine = model.predict(combine4predict)
        # predictions_Combine = model.predict(combine4predict[...,::-1])
        predicted_class_Combine = np.argmax(predictions_Combine)

        if predicted_class != predicted_class_Combine:
            successDetectionNum[j] += 1
    
for z in range(len(need_test)):
    print ('Adversarial Samples Detection Accuracy is: %.4f'  %(successDetectionNum[z]/len(attackSuccessIndex)))    
    

In [ ]:
print('image reversing')
successNum = [0,0,0,0,0,0]

for i in range(len(attackSuccessIndex)):
    img = x_val[i]*255

    img1 = np.expand_dims(img, axis=0)
    img1 = img1.astype('float32')

    img1 = preprocess_input(img1)

    img4predict = np.copy(img1)

    predictions = model.predict(img4predict)
    predicted_class = np.argmax(predictions)
    
    # cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "block5_conv3")
    cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "res5c_branch2c")
    
    combineImg = np.zeros((6, 224, 224, 3), dtype=np.float32)
    need_test = [0.00000001,0.1,0.2,0.3,0.4,0.5]
    for j in range(len(need_test)):
        combineImg[j] = cam_image * need_test[j] + img[...,::-1]

        #*******
        img2 = np.expand_dims(combineImg[j], axis=0)
        img2 = img2.astype('float32')
        
        #*******
        img2 = preprocess_input(img2)

        #*******
        combine4predict = np.copy(img2)

        predictions_Combine = model.predict(combine4predict)
        # predictions_Combine = model.predict(combine4predict[...,::-1])
        predicted_class_Combine = np.argmax(predictions_Combine)

        if predicted_class == predicted_class_Combine:
            successNum[j] += 1
    
for z in range(len(need_test)):
    print ('Detection Accuracy is: %.4f'  %(successNum[z]/len(attackSuccessIndex)))

    
    
    
successDetectionNum = [0,0,0,0,0,0]

for i in range(len(attackSuccessIndex)):
    # img = imgAdv[attackSuccessIndex[i]]
    img = imgAdv[attackSuccessIndex[i]]*255

    img1 = np.expand_dims(img, axis=0)
    img1 = img1.astype('float32')

    img1 = preprocess_input(img1)

    img4predict = np.copy(img1)

    predictions = model.predict(img4predict)
    predicted_class = np.argmax(predictions)
    
    # cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "block5_conv3")
    cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "res5c_branch2c")

    combineImg = np.zeros((6, 224, 224, 3), dtype=np.float32)
    need_test = [0.00000001,0.1,0.2,0.3,0.4,0.5]
    for j in range(len(need_test)):
        combineImg[j] = cam_image * need_test[j] + img[...,::-1]

        #*******
        img2 = np.expand_dims(combineImg[j], axis=0)
        img2 = img2.astype('float32')
        
        #*******
        img2 = preprocess_input(img2)

        #*******
        combine4predict = np.copy(img2)

        predictions_Combine = model.predict(combine4predict)
        # predictions_Combine = model.predict(combine4predict[...,::-1])
        predicted_class_Combine = np.argmax(predictions_Combine)

        if predicted_class != predicted_class_Combine:
            successDetectionNum[j] += 1
    
for z in range(len(need_test)):
    print ('Adversarial Samples Detection Accuracy is: %.4f'  %(successDetectionNum[z]/len(attackSuccessIndex)))  

In [ ]:
print('reversing cam image')
successNum = [0,0,0,0,0,0]

for i in range(len(attackSuccessIndex)):
    img = x_val[i]*255

    img1 = np.expand_dims(img, axis=0)
    img1 = img1.astype('float32')

    img1 = preprocess_input(img1)

    img4predict = np.copy(img1)

    predictions = model.predict(img4predict)
    predicted_class = np.argmax(predictions)
    
    # cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "block5_conv3")
    cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "res5c_branch2c")
    
    combineImg = np.zeros((6, 224, 224, 3), dtype=np.float32)
    need_test = [0.00000001,0.1,0.2,0.3,0.4,0.5]
    for j in range(len(need_test)):
        combineImg[j] = cam_image * need_test[j] + img

        #*******
        img2 = np.expand_dims(combineImg[j][:,:,::-1], axis=0)
        img2 = img2.astype('float32')
        
        #*******
        img2 = preprocess_input(img2)

        #*******
        combine4predict = np.copy(img2)

        predictions_Combine = model.predict(combine4predict)
        # predictions_Combine = model.predict(combine4predict[...,::-1])
        predicted_class_Combine = np.argmax(predictions_Combine)

        if predicted_class == predicted_class_Combine:
            successNum[j] += 1
    
for z in range(len(need_test)):
    print ('Detection Accuracy is: %.4f'  %(successNum[z]/len(attackSuccessIndex)))

    
    
    
successDetectionNum = [0,0,0,0,0,0]

for i in range(len(attackSuccessIndex)):
    # img = imgAdv[attackSuccessIndex[i]]
    img = imgAdv[attackSuccessIndex[i]]*255

    img1 = np.expand_dims(img, axis=0)
    img1 = img1.astype('float32')

    img1 = preprocess_input(img1)

    img4predict = np.copy(img1)

    predictions = model.predict(img4predict)
    predicted_class = np.argmax(predictions)
    
    # cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "block5_conv3")
    cam_image, heat_map = grad_cam(model, img4predict, predicted_class, "res5c_branch2c")

    combineImg = np.zeros((6, 224, 224, 3), dtype=np.float32)
    need_test = [0.00000001,0.1,0.2,0.3,0.4,0.5]
    for j in range(len(need_test)):
        combineImg[j] = cam_image * need_test[j] + img

        #*******
        img2 = np.expand_dims(combineImg[j][:,:,::-1], axis=0)
        img2 = img2.astype('float32')
        
        #*******
        img2 = preprocess_input(img2)

        #*******
        combine4predict = np.copy(img2)

        predictions_Combine = model.predict(combine4predict)
        # predictions_Combine = model.predict(combine4predict[...,::-1])
        predicted_class_Combine = np.argmax(predictions_Combine)

        if predicted_class != predicted_class_Combine:
            successDetectionNum[j] += 1
    
for z in range(len(need_test)):
    print ('Adversarial Samples Detection Accuracy is: %.4f'  %(successDetectionNum[z]/len(attackSuccessIndex)))  

In [ ]:
K.clear_session()

In [4]:
from PIL import Image
img111 = Image.fromarray(np.squeeze(imgAdv[0]).astype(np.uint8), 'RGB')
img111.save('oooooooooooook.png')

In [5]:
from PIL import Image
img111 = Image.fromarray(np.squeeze(x_val[0]).astype(np.uint8), 'RGB')
img111.save('oooo1ooook.png')